In [11]:
import numpy as np
import tensorflow.keras.backend as K

In [2]:
data_state = np.array([[-0.09490553,  0.01206422,  0.10296744, -0.09917992,  0.07588826,
       -0.07795104,  0.02191551, -0.13767721, -0.04881138, -0.06123666,
        0.05877448,  0.30867592, -0.19316114, -0.29431283, -0.2440839 ,
       -0.04894308, -0.52115715,  0.02771688,  0.03202315,  0.03136631,
       -0.16800883, -0.15039466, -0.08229536, -0.02807628,  0.09979199,
       -0.11563709, -0.03285349, -0.08969718,  0.02577355, -0.2774055 ,
        0.13139752, -0.3810906 , -0.21444742,  0.02582763,  0.2033777 ,
       -0.23192653,  0.16177502, -0.24736127,  0.04738998, -0.21546632,
       -0.08460043, -0.14324403,  0.09372416,  0.52963114, -0.49540657,
       -0.5108493 , -0.5829645 , -0.12721801, -1.3014983 ,  0.07168403,
        0.06727508,  0.06958196, -0.31567806, -0.34007278, -0.20917237,
       -0.05022753,  0.1695506 , -0.24195157, -0.08364195, -0.1422    ,
        0.06042047, -0.51047385,  0.29956988, -0.8082298 ]])

In [4]:
state_h = data_state[:,0:32]
state_c = data_state[:,32:]

In [8]:
wts=np.load('weight.npz')
weights=wts[wts.files[0]]

In [14]:
re_train=False
time_steps=1
n_input =3
num_units=32
n_classes=1
lr =0.0001

wi=weights[0][:,0:num_units]
wf=weights[0][:,num_units:num_units*2]
wc=weights[0][:,num_units*2:num_units*3]
wo=weights[0][:,num_units*3:num_units*4]
ui=weights[1][:,0:num_units]
uf=weights[1][:,num_units:num_units*2]
uc=weights[1][:,num_units*2:num_units*3]
uo=weights[1][:,num_units*3:num_units*4]

bi=weights[2][0:num_units]
bf=weights[2][num_units:num_units*2]

bc=weights[2][num_units*2:num_units*3]
bo=weights[2][num_units*3:num_units*4]

out_weights=weights[3]
out_bias=weights[4]

In [15]:
data_x=np.loadtxt('./TapData/data_raw_active.csv',delimiter=",")
data_y=np.loadtxt('./TapData/data_y_active.csv',delimiter=",")
data_y=data_y.reshape(len(data_y),1)
data_x=data_x[:,(6-n_input):6].reshape((len(data_x),3))
data_state = np.zeros([1,num_units*2],dtype='float')

In [16]:
x = data_x[1:2]

In [34]:
print(x.shape)
print(wi.shape)
print(np.dot(x,wi))

(1, 3)
(3, 32)
[[ 0.3389908   0.14714303  0.22978129 -0.12240854  0.03300347 -0.12367892
  -0.29690865 -0.13991258  0.33921028 -0.00242702 -0.17130661  0.16864227
   0.07261963 -0.10791417 -0.1986124  -0.06764315  0.21648077  0.01510706
   0.21877632 -0.02098285 -0.2419152   0.04788535 -0.00678408  0.06928057
   0.10669614 -0.18228838  0.1606234   0.12960549  0.02567338  0.36402938
   0.00940556 -0.01852135]]


In [36]:
print(np.transpose(np.dot(np.transpose(wi),np.transpose(x))))

[[ 0.3389908   0.14714303  0.22978129 -0.12240854  0.03300347 -0.12367892
  -0.29690865 -0.13991258  0.33921028 -0.00242702 -0.17130661  0.16864227
   0.07261963 -0.10791417 -0.1986124  -0.06764315  0.21648077  0.01510706
   0.21877632 -0.02098285 -0.2419152   0.04788535 -0.00678408  0.06928057
   0.10669614 -0.18228838  0.1606234   0.12960549  0.02567338  0.36402938
   0.00940556 -0.01852135]]


In [26]:
def hard_sigmoid(x):
    y=0.2 * x + 0.5
    y = np.clip(y,a_min=0,a_max=1)
    return y
    

In [29]:
x_i = (np.dot(x,wi)+bi)
x_f = (np.dot(x,wf)+bf)
x_c = (np.dot(x,wc)+bc)
x_o = (np.dot(x,wo)+bo)

i = hard_sigmoid(x_i+np.dot(state_h,ui))
f = hard_sigmoid(x_f+np.dot(state_h,uf))
c = f*state_c+np.tanh(x_c+np.dot(state_h,uc))
#tf.add(tf.multiply(f,state_c),K.tanh(tf.add(tf.add(x_c,bc),K.dot(state_h,uc))),name='output_c')
o = hard_sigmoid(x_o+np.dot(state_h,uo))
h = o*np.tanh(c)

#prediction=tf.nn.relu(tf.add(tf.matmul(h,out_weights), out_bias,name='output_forRelu'), name="output")
#state_out = tf.concat([h,c],axis=1,name='output_state')

In [30]:
h

array([[-0.26087013, -0.07923196,  0.10833303, -0.08073158,  0.02192846,
        -0.09816514, -0.00919418, -0.16754107, -0.27055097, -0.14367393,
         0.22230024,  0.35295213, -0.39103546, -0.34847275, -0.33515684,
        -0.0527762 , -0.5351516 ,  0.16447165,  0.07066188,  0.07050963,
        -0.19926658, -0.24653296, -0.21245113, -0.05774761,  0.1163515 ,
        -0.01639742, -0.14043776,  0.01351559,  0.17662127, -0.24192186,
         0.21953565, -0.46437971]])